In [ ]:
import numpy as np
import os
import subprocess as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.ticker import StrMethodFormatter

import matplotlib.colors as mcolors
custom_colors = ["white", "#DA8A67", "#A0522D", "#400000"]
custom_cmap = mcolors.LinearSegmentedColormap.from_list("custom_hot", custom_colors)

matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

def gettingFacets(filename,includeCoat='true'):
    exe = ["./getFacet-threePhase", filename, includeCoat]
    p = sp.Popen(exe, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = p.communicate()
    temp1 = stderr.decode("utf-8")
    temp2 = temp1.split("\n")
    segs = []
    skip = False
    if (len(temp2) > 1e2):
        for n1 in range(len(temp2)):
            temp3 = temp2[n1].split(" ")
            if temp3 == ['']:
                skip = False
                pass
            else:
                if not skip:
                    temp4 = temp2[n1+1].split(" ")
                    r1, z1 = np.array([float(temp3[1]), float(temp3[0])])
                    r2, z2 = np.array([float(temp4[1]), float(temp4[0])])
                    segs.append(((r1, z1),(r2, z2)))
                    segs.append(((-r1, z1),(-r2, z2)))
                    skip = True
    return segs
# ----------------------------------------------------------------------------------------------------------------------

def process_timestep(ti, folder, rmin, rmax, zmin, zmax, lw, placeFolder):
    t = 0.1 * ti
    place = f"{placeFolder}/intermediate/snapshot-{t:.4f}"
    name = f"{folder}/{int(t*1000):08d}.png"

    if not os.path.exists(place):
        print(f"{place} File not found!")
        return

    if os.path.exists(name):
        print(f"{name} Image present!")
        return

    segs1 = gettingFacets(place)
    segs2 = gettingFacets(place, 'false')

    print("got segments")
    if not segs1 and not segs2:
        print(f"Problem in the available file {place}")
        return

    # Plotting
    AxesLabel, TickLabel = 50, 20
    fig, ax = plt.subplots()
    fig.set_size_inches(19.20, 10.80)

    ax.plot([0, 0], [zmin, zmax], '-.', color='grey', linewidth=lw)
    ax.plot([rmin, rmin], [zmin, zmax], '-', color='black', linewidth=lw)
    ax.plot([rmin, rmax], [zmin, zmin], '-', color='black', linewidth=lw)
    ax.plot([rmin, rmax], [zmax, zmax], '-', color='black', linewidth=lw)
    ax.plot([rmax, rmax], [zmin, zmax], '-', color='black', linewidth=lw)

    line_segments = LineCollection(segs2, linewidths=4, colors='green', linestyle='solid')
    ax.add_collection(line_segments)
    line_segments = LineCollection(segs1, linewidths=4, colors='blue', linestyle='solid')
    ax.add_collection(line_segments)

    ax.set_aspect('equal')
    ax.set_xlim(rmin, rmax)
    ax.set_ylim(zmin, zmax)
    ax.set_title(f'$t/\\tau_\\gamma$ = {t:4.3f}', fontsize=TickLabel)
    ax.axis('off')

    plt.savefig(name, bbox_inches="tight")
    plt.close()

In [ ]:
nGFS = 550
Ldomain = 16
GridsPerR = 10
nr = int(GridsPerR * Ldomain)
rmin, rmax, zmin, zmax = [-3.0, 3.0, -Ldomain/2.0, Ldomain/2.0]
lw = 2
folder = 'Video_Noreduced'

if not os.path.isdir(folder):
    os.makedirs(folder)

for ti in range(1,nGFS):
    print(f"Processing {ti+1} of {nGFS} timesteps")
    process_timestep(ti, folder, rmin, rmax, zmin, zmax, lw, "testGravity_noreduced")
    print(f"Processed {ti+1} of {nGFS} timesteps")

In [ ]:
nGFS = 550
Ldomain = 16
GridsPerR = 10
nr = int(GridsPerR * Ldomain)
rmin, rmax, zmin, zmax = [-3.0, 3.0, -Ldomain/2.0, Ldomain/2.0]
lw = 2
folder = 'Video_Withreduced'

if not os.path.isdir(folder):
    os.makedirs(folder)

for ti in range(1,nGFS):
    print(f"Processing {ti+1} of {nGFS} timesteps")
    process_timestep(ti, folder, rmin, rmax, zmin, zmax, lw, "testGravity_Withreduced")
    print(f"Processed {ti+1} of {nGFS} timesteps")